# Skill estimation using Stan

## Potential Ideas 

    1. Compare the ability of your model to predict the winner of new (unseen) games to simple approaches, such as fraction of games won, number of games played, etc.
        - figure out ways to evaluate model performance: 
            (a) predict the winner of games in validation (Done)
            (b) predict the fraction of games won (Done)
            (c) predict game result: [? - ?] 
    
    2. Try evaluating how many games are required to accurately predict the players' skill levels / win probability by decreasing the amount of training data available and observing the performance. 
        - when processing data...may do pruning
            (a) vary # most recent games for each player --> look at distribution of date in train and valid first
            (b) vary max # opponents for each player to count
            (c) vary max # games with each oppo for each player to count 
    
    3. Try evaluating how quickly you can determine a new players' skill by either random game choices or carefully chosen games (matched based on estimated skill level).  You can leave a player out of the inference process entirely, then slowly add their games in and see how quickly you are able to learn their relative position.
        - may need to build a new model: 
            * input with skill levels for old players, games that new player(s) involved.
            * output the estimated skill level for new player(s). 
    
    4. Experiment with learning a more complex model, for example taking into account game features (player's selected character) or additional latent scores (such as offensive and defensive skill) along with a correspondingly more elaborate probability of win function.
        - Add weights for match date (the more recent one the more importance) -> related to Idea 2. 
        - # plays
        - race
        - addon
        - tournament-type
        * For those new features, may need to do hypothesis test later to verify their significance to the results.  

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

## Start Point: Use sample model

In [2]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [3]:
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [4]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    
#     for i in np.random.permutation(len(lines)):
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 

    return nplayers,playerA,playerB,wins

In [5]:
nplayers,playerA,playerB,wins = load_data()

In [6]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])

summary: 
# players 999
# games 4677
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
wins [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [7]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.3,
    'win':wins,
    'PA': playerA,
    'PB': playerB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [69]:
fit = sm.sampling(data=skill_data, iter=1000, chains=4)

In [70]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [71]:
samples['skill'].shape # 2*100 iterations? 999 players

(2000, 999)

In [72]:
samples['skill'].mean(0)

array([ 4.48720966e+00,  4.65973429e+00,  6.53511835e+00,  7.91035317e+00,
        6.43767562e+00,  1.54857279e+00,  4.74397142e+00,  1.54295123e+00,
        4.91299347e+00,  6.13785709e+00,  2.48626544e-01,  2.17244543e+00,
        2.50172374e+00, -6.37351131e-01,  4.05904456e+00,  2.63563031e+00,
       -1.17255877e+00, -1.71238734e+00, -4.84925538e-01,  4.77136647e+00,
       -5.03921977e-01,  2.48519009e+00,  3.77728552e+00,  4.84543478e+00,
        5.90940960e+00,  2.08599116e+00,  6.56073398e+00,  3.16906984e-01,
        4.81492203e+00,  2.74591122e+00,  4.62320893e+00, -1.28161602e+00,
        8.52762807e-01,  6.35426654e+00,  4.08801131e-01,  6.42795377e+00,
       -8.29093979e-01,  8.98409408e-01,  5.54759839e+00,  3.88376759e+00,
        8.86050646e-01,  4.87553445e+00,  2.53487857e+00,  4.09643587e+00,
       -1.92431535e-01,  3.98094000e+00,  5.01982405e+00,  5.10208811e+00,
        4.36354325e+00,  2.89537028e+00,  1.10702892e-01,  9.69728655e-01,
        2.18619150e+00, -

If we want to predict which player will win, we might use a direct estimator of that quantity based on the sample values:

In [73]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.4879440028879925


Remember to save the prediction!

In [74]:
with open('skill_hat.pkl', 'wb') as f: 
    pickle.dump(samples['skill'], f)

## Sample Model Evaluation

In [8]:
skill_hat = pickle.load(open('skill_hat.pkl', 'rb'))

In [9]:
def load_valid_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player  # id to name lookup


    # if validation, use all datapoints
    
    games = []
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        nplays[a,b] += 1
        nplays[b,a]+=1
        nwins[a,b] += aw
        nwins[b,a] += bw
    
    return nplayers, nplays, nwins, games


In [10]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data()

In [11]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 188014.0
(999, 999) 94291.0
games 0


In [14]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [15]:
loss, binary_loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [16]:
loss, binary_loss

(0.40859711344277694, 0.4447127906426102)

## Feature / Data reanalysis

Do that in a seperate notebook!

## New Model

Do that in a seperate notebook!